# Pulling data from the NHS BSA Open Data Portal (ODP) using R

In [1]:
# Load any packages
library(jsonlite)

The ODP https://opendata.nhsbsa.net/ has two programatic methods to access data from it...

* `datastore_search` e.g. https://opendata.nhsbsa.net/api/3/action/datastore_search?resource_id=EPD_201401&limit=5
* `datastore_search_sql` e.g. https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?sql=SELECT%20*%20FROM%20EPD_201401%20LIMIT%205

The following code demonstrates the process using the SQL style query. It is a more flexible way to access any data and easy if you already know some SQL (if not don't worry - the code is there for you to follow).

In [19]:
# Define the url for the API call
base_endpoint <- "https://opendata.nhsbsa.net/api/3/action"
action_method <- "/datastore_search_sql?sql=" # SQL

# Define the parameters for the SQL query
resource_name <- "EPD_202001"
pco_code <- "13T00" # Newcastle Gateshead CCG
bnf_chemical_substance <- "0407010H0" # Paracetamol

# Construct the SQL query
query <- paste0(
    "
    SELECT 
        * 
    FROM ", 
        resource_name, " 
    WHERE 
        1=1 
    AND pco_code = '", pco_code, "' 
    AND bnf_chemical_substance = '", bnf_chemical_substance, "'"
)

# Send API call and grab the response as a json
response <- jsonlite::fromJSON(paste0(
    base_endpoint,
    action_method, 
    URLencode(query) # Encode spaces in the url
))


$help
[1] "https://opendata.nhsbsa.net/api/3/action/help_show?name=datastore_search_sql"

$success
[1] TRUE

$result
$result$help
[1] "https://demo.ckan.org/api/3/action/help_show?name=datastore_search_sql"

$result$success
[1] "true"

$result$result
$result$result$records
            BNF_CODE TOTAL_QUANTITY POSTCODE YEAR_MONTH UNIDENTIFIED
1    0407010H0AAACAC            500  NE3 1EE     202001        FALSE
2    0407010H0AADSDS             60  NE4 6SS     202001        FALSE
3    0407010H0AABGBG            800  NE9 6SX     202001        FALSE
4    0407010H0AAAIAI            300  NE9 6SX     202001        FALSE
5    0407010H0AAAMAM           1800  NE8 4QR     202001        FALSE
6    0407010H0AAA7A7            100  NE6 1SG     202001        FALSE
7    0407010H0AAAMAM           3136  NE8 4QR     202001        FALSE
8    0407010H0AAAMAM            700  NE4 6SS     202001        FALSE
9    0407010H0AAAAAA            100  NE3 1EE     202001        FALSE
10   0407010H0AAAIAI            400 

The response from the API is held as a dictionary, you can view it by using the `print()` command below:

In [12]:
# Try to print some of the data we have... e.g. print(response), print(query)

ERROR: Error in parse(text = x, srcfile = src): <text>:6:0: unexpected end of input
4:     action_method, 
5:     URLencode(query))
  ^


Now we can use the `pandas` library to analyse the data in a tabular format. This is the most popular Python package for data manipulation and analysis.

In [20]:
# Extract records in the response to a dataframe
result_df = response$result$result$records

# View the first 6 rows of data
head(result_df)

,BNF_CODE,TOTAL_QUANTITY,POSTCODE,YEAR_MONTH,UNIDENTIFIED,PRACTICE_NAME,BNF_CHAPTER_PLUS_CODE,ACTUAL_COST,QUANTITY,REGIONAL_OFFICE_CODE,⋯,ADQUSAGE,PCO_CODE,REGIONAL_OFFICE_NAME,NIC,CHEMICAL_SUBSTANCE_BNF_DESCR,PRACTICE_CODE,PCO_NAME,AREA_TEAM_NAME,BNF_DESCRIPTION,ADDRESS_1
,<chr>,<dbl>,<chr>,<int>,<lgl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0407010H0AAACAC,500,NE3 1EE,202001,FALSE,ST OSWALDS PALLIATIVE CARE,04: Central Nervous System,9.68241,500,Y54,⋯,8.33333,13T00,NORTH OF ENGLAND,10.40,Paracetamol,Y00505,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 250mg/5ml oral suspension,ST OSWALDS HOSPICE
2,0407010H0AADSDS,60,NE4 6SS,202001,FALSE,MARIE CURIE HOSPICE,04: Central Nervous System,3.87813,60,Y54,⋯,10.00000,13T00,NORTH OF ENGLAND,4.05,Paracetamol,Y05648,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 500mg effervescent tablets,MARIE CURIE HOSPICE
3,0407010H0AABGBG,800,NE9 6SX,202001,FALSE,GATESHEAD EXTRA CARE,04: Central Nervous System,9.97996,200,Y54,⋯,13.33333,13T00,NORTH OF ENGLAND,10.68,Paracetamol,Y04833,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 250mg/5ml oral suspension sugar free,EMERGENCY CARE CENTRE
4,0407010H0AAAIAI,300,NE9 6SX,202001,FALSE,WALK-IN CENTRE GP IN-HOURS,04: Central Nervous System,3.82398,100,Y54,⋯,2.40000,13T00,NORTH OF ENGLAND,3.75,Paracetamol,Y04106,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 120mg/5ml oral suspension paediatric,WALK-IN CENTRE
5,0407010H0AAAMAM,1800,NE8 4QR,202001,FALSE,108 RAWLING ROAD(RAWLING ROAD PRACTICE),04: Central Nervous System,40.05622,100,Y54,⋯,300.00000,13T00,NORTH OF ENGLAND,42.84,Paracetamol,A85609,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 500mg tablets,108 RAWLING ROAD
6,0407010H0AAA7A7,100,NE6 1SG,202001,FALSE,NEWCASTLE GP IN MOLINEUX WIC,04: Central Nervous System,1.21887,100,Y54,⋯,0.80000,13T00,NORTH OF ENGLAND,1.19,Paracetamol,Y05671,NEWCASTLE GATESHEAD CCG,"CUMBRIA,NORTHUMB,TYNE & WEAR AREA",Paracetamol 120mg/5ml oral solution paediatric sugar free,MOLINEUX WALK-IN CENTRE


Next up we can utilise some of the inbuilt `pandas` plotting functionality to create some quick and easy visualisations

In [2]:
# Lets inspect the QUANTITY column
result_df.hist(column='QUANTITY')

# Can we try removing the background
result_df.hist(column='QUANTITY', grid=False)

# How about using more bins
result_df.hist(column='QUANTITY', grid=False, bins=50)

# What about one bin per value of QUANTITY
result_df.hist(
    column='QUANTITY', 
    grid=False, 
    bins=int(max(result_df['QUANTITY']))
)

# Lets see if QUANTITY varies by BNF_DESCRIPTION
result_df.hist(
    column='QUANTITY', 
    by='BNF_DESCRIPTION',
    grid=False, 
    bins=50,
    sharex=True, # All the rows share the same x axis
    layout=(18, 1), # 18 rows and one column
    figsize=(10, 20) # Make the graph big enough 
)

# We can see that BNF_DESCRIPTION contains different forms for the drugs... 
# why don't we limit this to 'tablet' and check again
tablet_df = result_df[result_df['BNF_DESCRIPTION'].str.contains('tablet')]
tablet_df.hist(
    column='QUANTITY', 
    by='BNF_DESCRIPTION',
    grid=False, 
    bins=int(max(tablet_df['QUANTITY'])), # Bin by each value of QUANTITY
    sharex=True,
    layout=(5, 1),
    figsize=(5, 10)
)

# We can see there are peaks for certain QUANTITY so lets examine the 10 most 
# common QUANITTY
tablet_df['QUANTITY'].value_counts().head(10)

ModuleNotFoundError: No module named 'pandas'

Now recreate the previous graph but for 'oral suspension' instead of 'tablet'

In [ ]:
# Try to create a DataFrame called oral_suspension_df and then produce a histogram from it